<a href="https://colab.research.google.com/github/ImPeekaboo/mytools/blob/main/Download-tools-V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####**Setup**

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>[Start] Colab Stay Alive</font>
# @markdown > This cell runs a JS code that will automatically press the reconnect button when you got disconnected due to idle.
# ================================================================ #

import IPython
from IPython.display import clear_output
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Clicked on the connect button"); 
     btn.click() 
     }
   
   btn = document.getElementById('connect')
   if (btn != null){
     console.log("Clicked on the reconnect button"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

clear_output()

In [ ]:
# ==============================#============================== #
# @markdown <font size="4">Run this cell if there is no methods working in Method 1.</font>
# @markdown > If you want to run other cell, stop this first and then run it again if you need it.
# ==============================#============================== #

import time
from IPython.display import clear_output


def loop():
    counter = 0
    second = 60

    while True:
        clear_output()

        print(f'This cell has been running for {counter} minute(s)...')

        counter += 1
        time.sleep(second)


if __name__ == '__main__':
    try:
        loop()
    except KeyboardInterrupt:
        clear_output()
        exit
    else:
        pass

In [ ]:
# ============================= FORM ============================= #
#@markdown <font size=5>← Install rclone</font>
build_version = "stable" #@param ["stable", "beta"]
#@markdown ---
automatically_clear_cell_output = True  # @param{type: "boolean"}
# ================================================================ #

import os
import IPython
from IPython.display import clear_output


if build_version == "stable":
    !curl https://rclone.org/install.sh | sudo bash
else:
    !curl https://rclone.org/install.sh | sudo bash -s beta

!sudo apt-get -y install fuse3

try:
    os.makedirs("/root/.config/rclone", exist_ok=True)
except OSError as error:
    pass

if automatically_clear_cell_output is True:
    clear_output()

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← Upload Configuration File </font>
# @markdown <p>If you already have an rclone configuration file, you can upload it by running this cell.</p>

# @markdown ---
MODE = "RCONFIG" # @param ['UTILS', 'RCONFIG', 'RCONFIG_append', "GENERATELIST"]
REMOTE = "" # @param {type:"string"}
QUERY_PATTERN = "" # @param {type:"string"}
# @markdown > <p><font size=2px>For those who are unable to upload local file: <a href="https://stackoverflow.com/a/58661947">StackOverflow</a></font></p>
# ================================================================ #

from os import path as _p
from IPython.display import HTML, clear_output

if not _p.exists("/root/.ipython/mixlab.py"):
    from shlex import split as _spl
    from subprocess import run  # nosec

    shellCmd = "wget -qq https://shirooo39.github.io/MiXLab/resources/mixlab.py \
                    -O /root/.ipython/mixlab.py"
    run(_spl(shellCmd))  # nosec

import importlib, mixlab
from google.colab import files  # pylint: disable=import-error #nosec
from mixlab import checkAvailable, runSh, rcloneConfigurationPath, prepareSession


def generateUploadList():
    prepareSession()
    if checkAvailable("/content/upload.txt"):
        runSh("rm -f upload.txt")
    runSh(
        f"rclone --config {rcloneConfigurationPath}/rclone.conf lsf {REMOTE}: --include '{QUERY_PATTERN}' --drive-shared-with-me --files-only --max-depth 1 > /content/upload.txt",
        shell=True,  # nosec
    )


def uploadLocalFiles():
    prepareSession()
    if MODE == "UTILS":
        filePath = "/root/.ipython/mixlab.py"
    elif MODE in ("RCONFIG", "RCONFIG_append"):
        filePath = f"{rcloneConfigurationPath}/rclone.conf"
    else:
        pass

    try:
      if checkAvailable(filePath):
        runSh(f"rm -f {filePath}")
      display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Upload rclone.conf from your local machine.</h2><br>"))
      uploadedFile = files.upload()
      fileNameDictKeys = uploadedFile.keys()
      fileNo = len(fileNameDictKeys)
      if fileNo > 1:
          for fn in fileNameDictKeys:
              runSh(f'rm -f "/content/{fn}"')
          return print("\nOnly upload one configuration file!")
      elif fileNo == 0:
          return print("\nFile upload cancelled.")
      elif fileNo == 1:
          for fn in fileNameDictKeys:
              if checkAvailable(f"/content/{fn}"):
                  if MODE == "RCONFIG_append":
                    import urllib
                    urllib.request.urlretrieve("https://shirooo39.github.io/MiXLab/resources/configurations/rclone/rclone.conf",
                                               "/usr/local/sessionSettings/rclone.conf")
                    with open(f"/content/{fn}", 'r+') as r:
                      new_data = r.read()
                      runSh(f'rm -f "/content/{fn}"')
                    with open(filePath, 'r+') as f:
                      old_data = f.read()
                      f.seek(0)
                      f.truncate(0)
                      f.write(old_data + new_data)
                    print("\nUpdate completed.")
                  else:
                    runSh(f'mv -f "/content/{fn}" {filePath}')
                    runSh(f"chmod 666 {filePath}")
                    runSh(f'rm -f "/content/{fn}"')
                    importlib.reload(mixlab)
                    !rm /content/upload.txt
                    clear_output()
                    print("rclone.conf has been uploaded to Colab!")
              return
      else:
          print("\nNo file is chosen!")
          return
    except:
      return print("\nFailed to upload!")


if MODE == "GENERATELIST":
    generateUploadList()
else:
    uploadLocalFiles()

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← Mount/Unmount rclone Drive (Optional)</font>
# @markdown Mount a remote drive as a local drive on a mountpoint.
# @markdown ---
Cache_Directory = "DISK" #@param ["RAM", "DISK"]
# ================================================================ #

import os
from IPython.display import HTML, clear_output
import uuid
import ipywidgets as widgets
from google.colab import output
import re

if not os.path.exists("/root/.ipython/mixlab.py"):
  from shlex import split as _spl
  from subprocess import run

  shellCmd = "wget -qq https://shirooo39.github.io/MiXLab/resources/mixlab.py \
                  -O /root/.ipython/mixlab.py"
  run(_spl(shellCmd))

from mixlab import (
    runSh,
    prepareSession,
    rcloneConfigurationPath,
)

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def ShowAC():
  clear_output(wait=True)
  display(
      widgets.HBox(
          [widgets.VBox(
              [widgets.HTML(
                  '''<h3 style="font-family:Trebuchet MS;color:#4f8bd6;margin-top:0px;">
                  Available drive to mount/unmount:</h3>
                  '''
                  ),
               mountNam]
               )
          ]
          )
      )
  
  display(HTML("<br>"), MakeButton("Mount", MountCMD, "primary"),
          MakeButton("Unmount", unmountCMD, "danger"))

prepareSession()
content = open(f"{rcloneConfigurationPath}/rclone.conf").read()
avCon = re.findall(r"^\[(.+)\]$", content, re.M)
mountNam = widgets.Dropdown(options=avCon)

if Cache_Directory == 'RAM':
  cache_path = '/dev/shm'
elif Cache_Directory == 'DISK':
  os.makedirs('/tmp', exist_ok=True)
  cache_path = '/tmp'

def MountCMD():
    mPoint = f"/content/drives/{mountNam.value}"
    os.makedirs(mPoint, exist_ok=True)
    cmd = rf"rclone mount {mountNam.value}: {mPoint}" \
      rf" --config {rcloneConfigurationPath}/rclone.conf" \
      ' --user-agent "Mozilla"' \
      ' --buffer-size 256M' \
      ' --transfers 10' \
      ' --vfs-cache-mode full' \
      ' --vfs-cache-max-age 0h0m1s' \
      ' --vfs-cache-poll-interval 0m1s' \
      f' --cache-dir {cache_path}' \
      ' --allow-other' \
      ' --daemon'

    if runSh(cmd, shell=True) == 0:
      print(f"The drive have been successfully mounted! - \t{mPoint}")
    else:
      print(f"Failed to mount the drive! - \t{mPoint}")

def unmountCMD():
  mPoint = f"/content/drives/{mountNam.value}"
  if os.system(f"fusermount -uz {mPoint}") == 0:
    runSh(f"rm -r {mPoint}")
    print(f"The drive have been successfully unmounted! - \t{mPoint}")
  else:
    runSh(f"fusermount -uz {mPoint}", output=True)

ShowAC()

####**Aria2c**

In [ ]:
import subprocess
import pathlib
import sys
from IPython.display import clear_output

def install_aria2():
    try:
        subprocess.run(["aria2c", "--version"], stdout=subprocess.DEVNULL, check=True)
    except FileNotFoundError:
        install_command = ['sudo apt update', 'sudo apt install -y aria2']
        for x in install_command:
            execute(x)
        clear_output()
    except Exception as error:
        print(f'[ERROR]: {error}')

def execute(command):
    try:
        with subprocess.Popen(
            command,
            shell=True,
            bufsize=1,
            universal_newlines=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT
        ) as popen:
            for line in popen.stdout:
                print(line.rstrip())
    except Exception as error:
        print(f'[ERROR]: {error}')

def main():
    install_aria2()

    # ==============================[ FORM ] ==============================
    #@markdown <font size="5">← [Start] aria2 CLI</font>
    download_url = "" #@param {type:"string"}
    download_location = "/content/drives/Onedrive" #@param {type:"string"}

    #@markdown ---
    #@markdown <font size="5"><center>⚙️ Basic Configuration ⚙️</center></font>
    dry_run = False #@param {type:"boolean"}
    check_integrity = True #@param {type:"boolean"}
    max_concurrent_downloads = 9 #@param {type:"slider", min:1, max:10, step:1}
    max_connection_per_server = "4" #@param [1, 2, 4, 8, 16, 24, 32]
    #@markdown ---
    #@markdown <font size="5"><center>⚙️ HTTP Configuration ⚙️</center></font>

    #@markdown ---
    #@markdown <font size="5"><center>⚙️ Bittorrent Configuration ⚙️</center></font>
    disable_seeding = True #@param {type:"boolean"}

    #@markdown ---
    #@markdown <font size="5"><center>⚙️ Advanced Configuration ⚙️</center></font>
    allow_overwrite = True #@param {type:"boolean"}
    always_resume_download = True #@param {type:"boolean"}
    enable_async_dns = True #@param {type:"boolean"}
    async_dns_server = "1.1.1.1" #@param {type:"string"}
    disable_ipv6 = True #@param {type:"boolean"}
    show_console_ouput = True #@param {type:"boolean"}
    show_console_readout = False #@param {type:"boolean"}
    summary_interval = 50 #@param {type:"slider", min:0, max:60, step:5}
    download_result_output = "default" #@param ["default", "full", "hide"]
    human_readable_output = True #@param {type:"boolean"}
    enable_terminal_color = True #@param {type:"boolean"}
    realtime_chunk_checksum = True #@param {type:"boolean"}
    file_allocation_method = "trunc" #@param ["none", "prealloc", "trunc", "falloc"]
    # ====================================================================
    flag_list = []
    # basic_flag = []
    # http_flag = []
    torrent_flag = []
    # advanced_flag = []

    if dry_run is True:
        flag_list.insert(0, '--dry-run=true')
    else:
        flag_list.insert(0, '--dry-run=false')
    if check_integrity is True:
        flag_list.insert(1, '--check-integrity=true')
    else:
        flag_list.insert(1, '--check-integrity=false')
    flag_list.insert(2, f'--max-concurrent-downloads={max_concurrent_downloads}')
    flag_list.insert(3, f'--max-connection-per-server={max_connection_per_server}')
    if allow_overwrite is True:
        flag_list.insert(4, '--allow-overwrite=true')
    else:
        flag_list.insert(4, '--allow-overwrite=false')
    if always_resume_download is True:
        flag_list.insert(5, '--always-resume=true')
    else:
        flag_list.insert(5, '--always-resume=false')
    if enable_async_dns is True:
        flag_list.insert(6, '--async-dns=true')
        if async_dns_server == '':
            flag_list.insert(7, async_dns_server)
        else:
            flag_list.insert(7, f'--async-dns-server="{async_dns_server}"')
    else:
        flag_list.insert(6, '--async-dns=false')
        flag_list.insert(7, '')
    if disable_ipv6 is True:
        flag_list.insert(8, '--disable-ipv6=true')
    else:
        flag_list.insert(8, '--disable-ipv6=false')
    if show_console_ouput is True:
        flag_list.insert(9, '--quiet=false')
    else:
        flag_list.insert(9, '--quiet=true')
    if show_console_readout is True:
        flag_list.insert(10, '--show-console-readout=true')
    else:
        flag_list.insert(10, '--show-console-readout=false')
    flag_list.insert(11, f'--summary-interval={summary_interval}')
    flag_list.insert(12, f'--download-result={download_result_output}')
    if human_readable_output is True:
        flag_list.insert(13, '--human-readable=true')
    else:
        flag_list.insert(13, '--human-readable=false')
    if enable_terminal_color is True:
        flag_list.insert(14, '--enable-color=true')
    else:
        flag_list.insert(14, '--enable-color=false')
    if realtime_chunk_checksum is True:
        flag_list.insert(15, '--realtime-chunk-checksum=true')
    else:
        flag_list.insert(15, '--realtime-chunk-checksum=false')
    flag_list.insert(16, f'--file-allocation={file_allocation_method}')
    if disable_seeding is True:
        torrent_flag.insert(0, '--seed-time=0')
    else:
        torrent_flag.insert(0, '--seed-time=120')

    if download_url == '':
        print('[ABORTED]: There is no download url given.')
    else:
        if download_location == '' or download_location == 'default':
            download_location = '/content/downloads'
            pathlib.Path('/content/downloads').mkdir(parents=True, exist_ok=True)
        else:
            pathlib.Path(download_location).mkdir(parents=True, exist_ok=True)

        execute(
            'aria2c ' \
            f'{flag_list[0]} ' \
            f'--dir="{download_location}" ' \
            f'{flag_list[1]} ' \
            f'{flag_list[2]} ' \
            f'{flag_list[3]} ' \
            f'{flag_list[4]} ' \
            f'{flag_list[5]} ' \
            f'{flag_list[6]} ' \
            f'{flag_list[7]} ' \
            f'{flag_list[8]} ' \
            f'{flag_list[9]} ' \
            f'{flag_list[10]} ' \
            f'{flag_list[11]} ' \
            f'{flag_list[12]} ' \
            f'{flag_list[13]} ' \
            f'{flag_list[14]} ' \
            f'{flag_list[15]} ' \
            f'{flag_list[16]} ' \
            f'{torrent_flag[0]} ' \
            f'"{download_url}"'
        )

if __name__ == '__main__':
    main()

####**Google Drive CLI**

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← [Clone] Google Drive CLI</font>
# ================================================================ #

import os
import IPython
from IPython.display import clear_output


GoogleDriveCLI_path1 = "/content/tools/GoogleDriveCLI"
GoogleDriveCLI_path2 = GoogleDriveCLI_path1 + "/gdrive"


def cloneGoogleDriveCLI():
    if os.path.exists(GoogleDriveCLI_path1 + "/gdrive"):
        pass
    else:
        # Big thanks to github user GrowtopiaJaw for providing a pre-compiled binary of Google Drive CLI.
        # https://github.com/GrowtopiaJaw/gdrive
        os.system("wget https://github.com/GrowtopiaJaw/gdrive/releases/download/v2.1.1/gdrive-linux-amd64")
        
        if not os.path.exists(GoogleDriveCLI_path1):
            # Big thanks to github user prasmussen for creating such an awesome tool.
            # https://github.com/prasmussen/gdrive
            os.makedirs("/content/tools/GoogleDriveCLI")

        os.system("mv /content/gdrive-linux-amd64 " + GoogleDriveCLI_path1 + "/gdrive")
        os.system("chmod +x " + GoogleDriveCLI_path1 + "/gdrive")


def initializeGoogleDriveCLI():
    if not os.path.exists(GoogleDriveCLI_path2):
        cloneGoogleDriveCLI()
        initializeGoogleDriveCLI()
    else:
        !"$GoogleDriveCLI_path2" "about"
        #clear_output(wait = True)


initializeGoogleDriveCLI()

In [ ]:
# ============================= FORM ============================= #
#@markdown <font size=5>← [Start] Google Drive CLI</font>
download_id = "" #@param{type:"string"}
#@markdown > Currently only support downloading a publicly shared file (a file, NOT a folder).
download_path = "/content/drives/Onedrive" #@param{type:"string"}
#@markdown > If left empty, the default download path will be used (/content/downloads).
# ================================================================ #

import os
import IPython
from IPython.display import HTML, clear_output


download_path_default = "/content/downloads"
GoogleDriveCLI_path1 = "/content/tools/GoogleDriveCLI"
GoogleDriveCLI_path2 = GoogleDriveCLI_path1 + "/gdrive"


if not os.path.exists(GoogleDriveCLI_path2):
        display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">❌ Unable to locate the required binary!<br>Make sure you have already run the cell above first!</h2></center><br>"))
else:
    if download_id == "":
        display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">❌ The download_id field is empty!</h2></center><br>"))
    else:
        if download_path == "":
            download_path = download_path_default
            if not os.path.exists(download_path):
                os.makedirs(download_path)
            else:
                pass
        elif not os.path.exists(download_path):
            os.makedirs(download_path)
        else:
            pass
        
        !"/content/tools/GoogleDriveCLI/gdrive" download --path "$download_path" "$download_id"
        
        if download_path is download_path_default:
            display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#00b24c;\">The download_path field is empty.<br>Download(s) are stored into the default download path (/content/downloads).</h2></center><br>"))
        else:
            display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#00b24c;\">Download(s) are stored into (" + download_path + ").</h2></center><br>"))

####**MEGA**

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← MEGA Login</font>
# @markdown <p>Please log in to MEGA first (only needed to use the Uploader).</p>
# ================================================================ #

from functools import wraps
import errno
import os
import signal
import subprocess
import shlex

class TimeoutError(Exception):
    pass


def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wraps(func)(wrapper)

    return decorator

if not os.path.exists("/root/.ipython/mixlab.py"):
    from subprocess import run
    from shlex import split

    shellCmd = "wget -qq https://raw.githubusercontent.com/shirooo39/MiXLab/master/resources/mixlab.py \
                    -O /root/.ipython/mixlab.py"
    run(split(shellCmd))
from mixlab import runSh

@timeout(10)
def runShT(args):
    return runSh(args, output=True)

# Installing MEGAcmd
if not os.path.exists("/usr/bin/mega-cmd"):
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    print("MEGA is installed.")
else:
    !pkill mega-cmd

# Enter MEGA credential
USERNAME = ""  # @param {type:"string"}
PASSWORD = ""  # @param {type:"string"}
if not (USERNAME == "" or PASSWORD == ""):
    try:
        runShT(f"mega-login {USERNAME} {PASSWORD}")
    except TimeoutError:
        runSh('mega-whoami', output=True)
else:
    print("Please enter your MEGA credential.")

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← [Start] MEGA Downloader</font>
URL = "" #@param {type:"string"}
OUTPUT_PATH = "/content/drives/Onedrive" #@param {type:"string"}
# @markdown > URL: is the MEGA link you want to download (ex: mega.nz/file/file_link#decryption_key)</br>OUTPUT_PATH: is where to store the downloaded file(s) (ex: /content/downloads/)
# ================================================================ #

import sys, os, urllib.request
import time
import subprocess
import contextlib
from IPython.display import clear_output

HOME = os.path.expanduser("~")
if not os.path.exists(f"{HOME}/.ipython/mixlab.py"):
    hCode = "https://raw.githubusercontent.com/shirooo39/" \
                "MiXLab/master/resources/mixlab.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/mixlab.py")

from mixlab import (
    runSh,
    loadingAn,
)

if not OUTPUT_PATH:
  os.makedirs("downloads", exist_ok=True)
  OUTPUT_PATH = "downloads"
# Installing MEGAcmd
if not os.path.exists("/usr/bin/mega-cmd"):
    loadingAn()
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    print("MEGA is installed.")
    clear_output()

# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']

def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out

def transfare():
    import codecs
    decoder = codecs.getincrementaldecoder("UTF-8")()
    cmd = ["mega-get", URL, OUTPUT_PATH]
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        print(line)
        
transfare()

####**qBittorrent**

In [ ]:

# ============================= FORM ============================= #
# @markdown <font size=5>← [Start] rclone</font>
Mode = "Move" #@param ["Copy", "Move", "Sync", "Checker", "Deduplicate", "Remove Empty Directories", "Empty Trash"]
Source = "" #@param {type:"string"}
Destination = "/content/drives/Onedrive" #@param {type:"string"}

#@markdown ---
#@markdown <center><h3><font color="#3399ff"><b>⚙️ Global Configuration ⚙️</b></font></h3></center>
Extra_Arguments = "" #@param {type:"string"}
Compare = "Size & Mod-Time" #@param ["Size & Mod-Time", "Size & Checksum", "Only Mod-Time", "Only Size", "Only Checksum"]
Checkers = 29 #@param {type:"slider", min:1, max:40, step:1}
Transfers = 13 #@param {type:"slider", min:1, max:20, step:1}
Dry_Run = False #@param {type:"boolean"}
Do_not_cross_filesystem_boundaries = False
Do_not_update_modtime_if_files_are_identical = False #@param {type:"boolean"}
Google_Drive_optimization = False #@param {type:"boolean"}
Large_amount_of_files_optimization = False #@param {type:"boolean"}
Simple_Ouput = True #@param {type:"boolean"}
Skip_all_files_that_exist = False #@param {type:"boolean"}
Skip_files_that_are_newer_on_the_destination = False #@param {type:"boolean"}
Output_Log_File = "OFF" #@param ["OFF", "NOTICE", "INFO", "ERROR", "DEBUG"]

#@markdown <br><center><h3><font color="#3399ff"><b>↪️ Sync Configuration ↩️</b></font></h3></center>
Sync_Mode = "Delete during transfer" #@param ["Delete during transfer", "Delete before transfering", "Delete after transfering"]
Track_Renames = False #@param {type:"boolean"}

#@markdown <br><center><h3><font color="#3399ff"><b>💞 Deduplicate Configuration 💞</b></font></h3></center>
Deduplicate_Mode = "Interactive" #@param ["Interactive", "Skip", "First", "Newest", "Oldest", "Largest", "Rename"]
Deduplicate_Use_Trash = True #@param {type:"boolean"}

#@markdown ---
automatically_clear_cell_output = False  # @param{type: "boolean"}
# ================================================================ #

##### Importing the needed modules
import os
import IPython
from IPython.display import HTML, clear_output


##### Variable Declaration
# Optimized for Google Colaboratory
os.environ["bufferC"] = "--buffer-size 96M"

if Compare == "Size & Checksum":
    os.environ["compareC"] = "-c"
elif Compare == "Only Mod-Time":
    os.environ["compareC"] = "--ignore-size"
elif Compare == "Only Size":
    os.environ["compareC"] = "--size-only"
elif Compare == "Only Checksum":
    os.environ["compareC"] = "-c --ignore-size"
else:
    os.environ["compareC"] = ""

os.environ["sourceC"] = Source
os.environ["destinationC"] = Destination
os.environ["transfersC"] = "--transfers "+str(Transfers)
os.environ["checkersC"] = "--checkers "+str(Checkers)

if Skip_files_that_are_newer_on_the_destination == True:
    os.environ["skipnewC"] = "-u"
else:
    os.environ["skipnewC"] = ""
  
if Skip_all_files_that_exist == True:
    os.environ["skipexistC"] = "--ignore-existing"
else:
    os.environ["skipexistC"] = ""
  
if Do_not_cross_filesystem_boundaries == True:
    os.environ["nocrossfilesystemC"] = "--one-file-system"
else:
    os.environ["nocrossfilesystemC"] = ""
  
if Do_not_update_modtime_if_files_are_identical == True:
    os.environ["noupdatemodtimeC"] = "--no-update-modtime"
else:
    os.environ["noupdatemodtimeC"] = ""

if Large_amount_of_files_optimization == True:
    os.environ["filesoptimizeC"] = "--fast-list"
else:
    os.environ["filesoptimizeC"] = ""
  
if Google_Drive_optimization == True:
    os.environ["driveoptimizeC"] = "--drive-chunk-size 32M --drive-acknowledge-abuse --drive-keep-revision-forever"
else:
    os.environ["driveoptimizeC"] = ""
  
if Dry_Run == True:
    os.environ["dryrunC"] = "-n"
else:
    os.environ["dryrunC"] = ""
  
if Output_Log_File != "OFF":
    os.environ["statsC"] = "--log-file=/root/.rclone_log/rclone_log.txt"
else:
    if Simple_Ouput == True:
        os.environ["statsC"] = "-v --stats-one-line --stats=5s"
    else:
        os.environ["statsC"] = "-v --stats=5s"
  
if Output_Log_File == "INFO":
    os.environ["loglevelC"] = "--log-level INFO"
elif Output_Log_File == "ERROR":
    os.environ["loglevelC"] = "--log-level ERROR"
elif Output_Log_File == "DEBUG":
    os.environ["loglevelC"] = "--log-level DEBUG"
else:
    os.environ["loglevelC"] = ""

os.environ["extraC"] = Extra_Arguments

if Sync_Mode == "Delete during transfer":
    os.environ["syncmodeC"] = "--delete-during"
elif Sync_Mode == "Delete before transfering":
    os.environ["syncmodeC"] = "--delete-before"
elif Sync_Mode == "Delete after transfering":
    os.environ["syncmodeC"] = "--delete-after"
  
if Track_Renames == True:
    os.environ["trackrenamesC"] = "--track-renames"
else:
    os.environ["trackrenamesC"] = ""
  
if Deduplicate_Mode == "Interactive":
    os.environ["deduplicateC"] = "interactive"
elif Deduplicate_Mode == "Skip":
    os.environ["deduplicateC"] = "skip"
elif Deduplicate_Mode == "First":
    os.environ["deduplicateC"] = "first"
elif Deduplicate_Mode == "Newest":
    os.environ["deduplicateC"] = "newest"
elif Deduplicate_Mode == "Oldest":
    os.environ["deduplicateC"] = "oldest"
elif Deduplicate_Mode == "Largest":
    os.environ["deduplicateC"] = "largest"
elif Deduplicate_Mode == "Rename":
    os.environ["deduplicateC"] = "rename"
  
if Deduplicate_Use_Trash == True:
    os.environ["deduplicatetrashC"] = ""
else:
    os.environ["deduplicatetrashC"] = "--drive-use-trash=false"


##### rclone Execution
if Output_Log_File != "OFF" and Mode != "Config":
    !mkdir -p -m 666 /root/.rclone_log/
    display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Logging enabled, rclone will no longer display any output on the terminal.<br>Please wait until the cell stop by itself.</h2></center><br>"))

if Mode == "Copy":
    !rclone --config=/root/.config/rclone/rclone.conf copy "$sourceC" "$destinationC" $transfersC $checkersC $statsC $loglevelC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
elif Mode == "Move":
    !rclone --config=/root/.config/rclone/rclone.conf move "$sourceC" "$destinationC" $transfersC $checkersC $statsC $loglevelC --delete-empty-src-dirs $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
elif Mode == "Sync":
    !rclone --config=/root/.config/rclone/rclone.conf sync "$sourceC" "$destinationC" $transfersC $checkersC $statsC $loglevelC $syncmodeC $trackrenamesC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
elif Mode == "Checker":
    !rclone --config=/root/.config/rclone/rclone.conf check "$sourceC" "$destinationC" $checkersC $statsC $loglevelC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
elif Mode == "Deduplicate":
    !rclone --config=/root/.config/rclone/rclone.conf dedupe "$sourceC" $checkersC $statsC $loglevelC --dedupe-mode $deduplicateC $deduplicatetrashC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
elif Mode == "Remove Empty Directories":
    !rclone --config=/root/.config/rclone/rclone.conf rmdirs "$sourceC" $statsC $loglevelC $dryrunC $extraC
elif Mode == "Empty Trash":
    !rclone --config=/root/.config/rclone/rclone.conf cleanup "$sourceC" $statsC $loglevelC $dryrunC $extraC


##### Log Output
if Output_Log_File != "OFF" and Mode != "Config":

    ##### Rename log file and output settings.
    !mv /root/.rclone_log/rclone_log.txt /root/.rclone_log/rclone_log_$(date +%Y-%m-%d_%H.%M.%S).txt
    with open("/root/.rclone_log/" + Mode + "_settings.txt", "w") as f:
        f.write("Mode: " + Mode + \
            "\nCompare: " + Compare + \
            "\nSource: \"" + Source + \
            "\"\nDestination: \"" + Destination + \
            "\"\nTransfers: " + str(Transfers) + \
            "\nCheckers: " + str(Checkers) + \
            "\nSkip files that are newer on the destination: " + str(Skip_files_that_are_newer_on_the_destination) + \
            "\nSkip all files that exist: " + str(Skip_all_files_that_exist) + \
            "\nDo not cross filesystem boundaries: " + str(Do_not_cross_filesystem_boundaries) + \
            "\nDo not update modtime if files are identical: " + str(Do_not_update_modtime_if_files_are_identical) + \
            "\nDry-Run: " + str(Dry_Run) + \
            "\nOutput Log Level: " + Output_Log_File + \
            "\nExtra Arguments: \"" + Extra_Arguments + \
            "\"\nSync Moden: " + Sync_Mode + \
            "\nTrack Renames: " + str(Track_Renames) + \
            "\nDeduplicate Mode: " + Deduplicate_Mode + \
            "\nDeduplicate Use Trash: " + str(Deduplicate_Use_Trash))

    ##### Compressing log file.
    !rm -f /root/rclone_log.zip
    !zip -r -q -j -9 /root/rclone_log.zip /root/.rclone_log/
    !rm -rf /root/.rclone_log/
    !mkdir -p -m 666 /root/.rclone_log/

    ##### Send Log
    if os.path.isfile("/root/rclone_log.zip") == True:
        try:
            files.download("/root/rclone_log.zip")
            !rm -f /root/rclone_log.zip
            display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#446785;\">Sending log to your browser...</h2><br></center>"))
        except:
            !mv /root/rclone_log.zip /content/rclone_log_$(date +%Y-%m-%d_%H.%M.%S).zip
            display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#446785;\">You can use file explorer to download the log file.</h2><br><img src=\"https://minormole.github.io/RcloneLab/res/rclonelab/01.png\"><br></center>"))
    else:
        clear_output()
        display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">There is no log file.</h2><br></center>"))
    

### Operation has been successfully completed.
if Mode != "Config":
    display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#00b24c;\">✅ Operation has been successfully completed.</h2><br></center>"))


##### Automatically clear terminal output if the checkbox's value on the top is set to True.
if automatically_clear_cell_output is True:
    clear_output()
else:
    pass##### Automatically clear terminal output if the checkbox's value on the top is set to True.
if automatically_clear_cell_output is True:
    clear_output()
else:
    pass

In [ ]:

# ============================= FORM ============================= #
# @markdown <font size=5>← [Start] qBittorrent</font>
# @markdown <p>MiXLab is now using VueTorrent as the default qBittorrent WebUI.</p>
#QBITTORRENT_VARIANT = "official" #@param ["official", "unofficial"]
## @markdown ---
## @markdown <p>qBittorrent Default Credential</p>
## @markdown > Username: admin</br>Password: adminadmin
## @markdown ---
USE_FREE_TOKEN = True
TOKEN = ""
REGION = "US"
Tunneling = "argo_tunnel_(cloudflare)" #@param ["argo_tunnel_(cloudflare)", "localhost.run", "ngrok"]

if Tunneling == "argo_tunnel_(cloudflare)":
    PORT_FORWARD = "argotunnel"
elif Tunneling == "localhost.run":
    PORT_FORWARD = "localhost"
elif Tunneling == "ngrok":
    PORT_FORWARD = "ngrok"
# ================================================================ #

import os, psutil, time, urllib.request
from IPython.display import HTML, clear_output


!wget -P /content/qBittorrent/tmp https://raw.githubusercontent.com/shirooo39/MiXLab/master/resources/packages/qbittorrent/vuetorrent.zip
!unzip /content/qBittorrent/tmp/vuetorrent.zip -d /content/qBittorrent/tmp
!mv /content/qBittorrent/tmp/vuetorrent/ /content/qBittorrent/WebUI
clear_output()


HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/mixlab.py"):
    hCode = "https://raw.githubusercontent.com/shirooo39/" \
            "MiXLab/master/resources/mixlab.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/mixlab.py")

from mixlab import (
    checkAvailable,
    loadingAn,
    PortForward_wrapper,
    runSh,
    displayUrl,
    findProcess
)

#Note: need to locate where the WebUI is extracted into and then remove it
#      in order to use the proper WebUI for the Official or Unofficial version of qBittorrent
#runSh("rm -f /etc/apt/sources.list.d/qbittorrent-team-ubuntu-qbittorrent-stable-bionic.list")
#runSh("rm -f /etc/apt/sources.list.d/qbittorrent-team-ubuntu-qbittorrent-stable-bionic.list.save")
#runSh("rm -f /etc/apt/sources.list.d/poplite-ubuntu-qbittorrent-enhanced-bionic.list")
#runSh("rm -f /etc/apt/sources.list.d/poplite-ubuntu-qbittorrent-enhanced-bionic.list.save")
#runSh("rm -f /usr/bin/qbittorrent")
#runSh("rm -f /usr/bin/qbittorrent-nox")
#runSh("sudo apt-get purge --auto-remove qbittorrent-nox ")
#clear_output()

def addUtils():
    if not checkAvailable("/usr/local/sessionSettings"):
        runSh("mkdir -p -m 777 /usr/local/sessionSettings")
    if not checkAvailable("/content/upload.txt"):
        runSh("touch /content/upload.txt")
    if not checkAvailable("checkAptUpdate.txt", userPath=True):
        runSh("apt update -qq -y")
        runSh("apt-get install -y iputils-ping")

def configTimezone(auto=True):
    if checkAvailable("timezone.txt", userPath=True):
        return
    if not auto:
        runSh("sudo dpkg-reconfigure tzdata")
    else:
        runSh("sudo ln -fs /usr/share/zoneinfo/Asia/Ho_Chi_Minh /etc/localtime")
        runSh("sudo dpkg-reconfigure -f noninteractive tzdata")

def uploadQBittorrentConfig():
    if checkAvailable("updatedQBSettings.txt", userPath=True):
        return
    runSh(
        "mkdir -p -m 666 /content/qBittorrent /root/.qBittorrent_temp /root/.config/qBittorrent"
    )
    runSh(
        "wget -qq https://raw.githubusercontent.com/shirooo39/MiXLab/master/resources/configurations/qbittorrent/qBittorrent.conf \
            -O /root/.config/qBittorrent/qBittorrent.conf"
    )

def prepareSession():
    if checkAvailable("ready.txt", userPath=True):
        return
    else:
        addUtils()
        configTimezone()
        uploadQBittorrentConfig()

def installQBittorrent():
    if checkAvailable("/usr/bin/qbittorrent-nox"):
        return
    else:
#        if QBITTORRENT_VARIANT == "official":
            try:
#                if checkAvailable("/etc/apt/sources.list.d/poplite-ubuntu-qbittorrent-enhanced-bionic.list"):
#                    runSh("rm /etc/apt/sources.list.d/poplite-ubuntu-qbittorrent-enhanced-bionic.list")
#                elif checkAvailable("/etc/apt/sources.list.d/poplite-ubuntu-qbittorrent-enhanced-bionic.list.save"):
#                    runSh("rm /etc/apt/sources.list.d/poplite-ubuntu-qbittorrent-enhanced-bionic.list.save")
#                else:
                    runSh("sudo add-apt-repository ppa:qbittorrent-team/qbittorrent-stable")
                    runSh("sudo apt-get update")
                    runSh("sudo apt install qbittorrent-nox")
            except:
                raise Exception('Failed to install qBittorrent!')
#        elif QBITTORRENT_VARIANT == "unofficial":
#            try:
#                if checkAvailable("/etc/apt/sources.list.d/qbittorrent-team-ubuntu-qbittorrent-stable-bionic.list"):
#                    runSh("rm /etc/apt/sources.list.d/qbittorrent-team-ubuntu-qbittorrent-stable-bionic.list")
#                elif checkAvailable("/etc/apt/sources.list.d/qbittorrent-team-ubuntu-qbittorrent-stable-bionic.list.save"):
#                    runSh("rm /etc/apt/sources.list.d/qbittorrent-team-ubuntu-qbittorrent-stable-bionic.list.save")
#                else:
#                    runSh("sudo add-apt-repository ppa:poplite/qbittorrent-enhanced")
#                    runSh("sudo apt-get update")
#                    runSh("sudo apt-get install qbittorrent-enhanced qbittorrent-enhanced-nox")
#            except:
#                raise Exception('Failed to install qBittorrent!')

def startQBService():
    prepareSession()
    installQBittorrent()
    if not findProcess("qbittorrent-nox", "-d --webui-port"):
        runSh(f"qbittorrent-nox -d --webui-port={QB_Port}")
        time.sleep(1)

QB_Port = 10001
loadingAn()
startQBService()

clear_output()

server = PortForward_wrapper(PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['qbittorrent', QB_Port, 'http']], REGION.lower, [f"{HOME}/.ngrok2/qbittorrent.yml", 4088]).start('qbittorrent', displayB=False)
displayUrl(server, pNamU='qBittorrent : ')

####**libtorrent**

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← Install libtorrent</font>
# ================================================================ #

from IPython.display import clear_output

!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!apt install python3-libtorrent

import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

clear_output()

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← Add Torrent from File</font>
# @markdown <p><font size=5><center>How to change the download location:</center></font></br><font size=4>1. Double click the cell to show its code</br>2. Find this line: "save_path": "/content/downloads",</br>3. Change /content/downloads to your path</font></p>
# @markdown > You can run this cell as many time as you want.
# ================================================================ #

import os
from IPython.display import clear_output
from google.colab import files

if os.path.exists("/content/downloads"):
  pass
else:
  os.mkdir("/content/downloads")

source = files.upload()
params = {
    "save_path": "/content/drives/Onedrive",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

clear_output()

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← Add Torrent from Magnet Link</font>
# @markdown <p><font size=5><center>How to change the download location:</center></font></br><font size=4>1. Double click the cell to show its code</br>2. Find this line: params = {"save_path": "/content/downloads"}</br>3. Change /content/downloads to your path</font></p>
# @markdown > You can run this cell as many time as you want.
# ================================================================ #

import os
from IPython.display import clear_output

if os.path.exists("/content/downloads"):
  pass
else:
  os.mkdir("/content/downloads")

params = {"save_path": "/content/drives/Onedrive"}

while True:
    magnet_link = input("Paste the magnet link here or type exit to stop:\n")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )

clear_output()

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← [Start] libtorrent</font>
# ================================================================ #

import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)